In [1]:
import gym

env = gym.make('Pong-v0',render_mode='rgb_array')
observation = env.reset()


/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [2]:
# Render the game state
frame = env.render()
import cv2

# Convert to grayscale and resize to a lower resolution
gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
small_frame = cv2.resize(gray_frame, (84, 84))  # Resize to 84x84 for simplicity


/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [3]:
# Flatten image for the LLM input (example)
input_to_llm = small_frame.flatten()


In [4]:
description = "Ball is at (50, 30), player paddle is at y = 40, opponent paddle is at y = 60."
# action_text = predict_action_llm(description)
# print(f"LLM suggests action: {action_text}")


In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Check if MPS is available and use it
device = torch.device("mps" if torch.has_mps else "cpu")
model.to(device)

def predict_action_llm(description):
    input_ids = tokenizer.encode(description, return_tensors='pt', truncation=True).to(device)  # No max_length here
    output = model.generate(input_ids, max_new_tokens=30)  # Control the output length
    action_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return action_text

device = torch.device("mps" if torch.has_mps else "cpu")

print(f"MPS available: {torch.has_mps}")


MPS available: True


/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_3013/3070159875.py:8: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")
/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_3013/3070159875.py:17: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")
/var/folders/kh/kdb6f8j523s5fh4syx6bp5bh0000gn/T/ipykernel_3013/3070159875.py:19: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  print(f"MPS available: {torch.has_mps}")


In [22]:
def describe_game_state(frame):
    # Extract features of the frame: ball position, player paddle position, etc.
    # For this example, I'm using placeholder values for positions.
    ball_x, ball_y = 50, 30
    player_paddle_y = 40
    opponent_paddle_y = 60
    
    # Return a text description of the game state
    description = (f"Ball is at ({ball_x}, {ball_y}), "
                   f"player paddle is at y = {player_paddle_y}, "
                   f"opponent paddle is at y = {opponent_paddle_y}.")
    return description


In [23]:
def map_action(action_text):
    if "up" in action_text.lower():
        return 2  # Move paddle up
    elif "down" in action_text.lower():
        return 3  # Move paddle down
    else:
        return 0  # Stay still


In [24]:
# Example: Call LLM decision every few frames, instead of every frame
step = 0
while True:
    step += 1
    frame = env.render()
    
    if step % 10 == 0:  # Call LLM only every 10 frames
        description = describe_game_state(frame)
        action_text = predict_action_llm(description)
        action = map_action(action_text)
    else:
        action = 0  # Default action when not calling the LLM
    
    observation, reward, done, info, truncation = env.step(action)
    print(reward)


    
    if done:
        break


0.0
